# A First Look at the Data

In this notebook we will have a first look at the H.E.S.S. public data set. More information on this data set can be found here: [https://www.mpi-hd.mpg.de/HESS/pages/dl3-dr1/](https://www.mpi-hd.mpg.de/HESS/pages/dl3-dr1/) . 

We will use gammapy ([https://gammapy.org/](https://gammapy.org/)) which heavily depends on astropy ([https://www.astropy.org/](https://www.astropy.org/)). We will see some data manipulation with numpy ([https://numpy.org/](https://numpy.org/)) and data presentation using matplotlib ([https://matplotlib.org/](https://matplotlib.org/)) and pyplot ([https://matplotlib.org/stable/tutorials/pyplot.html](https://matplotlib.org/stable/tutorials/pyplot.html)).

## imports

Let's start with importing these modules.

In [ ]:
import gammapy

In [ ]:
gammapy.__version__

In [ ]:
import numpy as np

In [ ]:
np.__version__

In [ ]:
import matplotlib
matplotlib.__version__

In [ ]:
import matplotlib.pyplot as plt

## Download the data

Gammapy integrates the H.E.S.S. public data set in its tutorials. We can use gammapy to download the data easily.

In [ ]:
from gammapy.utils.check import check_tutorials_setup

In [ ]:
check_tutorials_setup()

In [ ]:
import os

In [ ]:
os.environ['GAMMAPY_DATA'] = 'gammapy-data/1.2'

All the essential information about the observations is found in a DataStore.

In [ ]:
from gammapy.data import DataStore

In [ ]:
data_store = DataStore.from_dir("$GAMMAPY_DATA/hess-dl3-dr1")

In [ ]:
data_store.info()

Expected output:

```
Data store:
HDU index table:
BASE_DIR: gammapy-data/1.2/hess-dl3-dr1
Rows: 630
OBS_ID: 20136 -- 47829
HDU_TYPE: ['aeff', 'bkg', 'edisp', 'events', 'gti', 'psf']
HDU_CLASS: ['aeff_2d', 'bkg_3d', 'edisp_2d', 'events', 'gti', 'psf_table']


Observation table:
Observatory name: 'N/A'
Number of observations: 105
```

## DataStore

In [ ]:
data_store.obs_table

This is an astropy.Table ([https://docs.astropy.org/en/stable/table/index.html](https://docs.astropy.org/en/stable/table/index.html)). Have look, these tables are very powerful.

In [ ]:
data_store.obs_table.colnames

In [ ]:
np.unique(data_store.obs_table['OBJECT'])

In [ ]:
data_store.hdu_table

## Run Selection

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord, Angle

We set the source position. frame='icrs' indicates that we are using coordinates in right ascension and declination.

In [ ]:
source_pos = SkyCoord(83.633*u.deg, 22.014*u.deg, frame='icrs')

In [ ]:
selectradius = 2.5*u.deg

In [ ]:
conesearch = data_store.obs_table.select_sky_circle(source_pos, selectradius)

In [ ]:
conesearch

In [ ]:
runlist = conesearch['OBS_ID'].value

In [ ]:
print(runlist)

In [ ]:
len(runlist)

In [ ]:
observations = data_store.get_observations(runlist)

In [ ]:
observations.ids

In [ ]:
obs = observations[0]

In [ ]:
obs.obs_id

In [ ]:
obs

## Event Lists

The most information is found in the event list. It is simply a list of recorded events. Remember, we measure indivual gamma-ray photons.

In [ ]:
from gammapy.data import EventList

In [ ]:
events = obs.events

In [ ]:
events

In [ ]:
events.peek()

In [ ]:
events.table

In [ ]:
plt.hist2d(events.table['RA'], 
           events.table['DEC'], 
           #bins = [100,100]
          )

In [ ]:
plt.hist(events.table['ENERGY'],
         #log = True,
         #range = [0.1,100],
         #bins = 100,
        )

## Theta$^2$ Plot

### on-source counts

In [ ]:
events.radec

In [ ]:
first_event = events.radec[0]

In [ ]:
first_event

In [ ]:
first_event.separation(source_pos)

In [ ]:
theta2 = events.radec.separation(source_pos)**2

In [ ]:
print(theta2)

In [ ]:
ret = plt.hist(theta2.value, 
#               range = [0,0.1], 
#               bins = 50
              )

plt.xlabel('$\\theta^2$ [deg$^2$]')

#plt.savefig('Theta2_on.svg')

In [ ]:
ret

In [ ]:
n = ret[0]
x = ret[1]

In [ ]:
print(x)

In [ ]:
x[1:]<0.01

In [ ]:
n[ x[1:]<0.01 ]

In [ ]:
oncounts = n[x[1:]<0.01].sum()

In [ ]:
oncounts

### off-source counts

In [ ]:
print(obs)

In [ ]:
pointing_pos = obs.pointing.fixed_icrs

In [ ]:
pointing_pos

In [ ]:
separation = pointing_pos.separation(source_pos)
print (separation)

In [ ]:
position_angle = pointing_pos.position_angle(source_pos)
print (position_angle.to(u.deg))

In [ ]:
offpos = pointing_pos.directional_offset_by( position_angle+180*u.deg, separation)

In [ ]:
offpos

In [ ]:
theta2_off = events.radec.separation(offpos)**2

In [ ]:
plt.hist(theta2.value, range = [0,0.1], bins = 50)
ret_off = plt.hist(theta2_off.value, range = [0,0.1], bins = 50, alpha = 0.5)

plt.xlabel('$\\theta^2$ [deg$^2$]')

#plt.savefig('Theta2_onoff.svg')

In [ ]:
n_off = ret_off[0]
offcounts = n_off[x[1:]<0.01].sum()

In [ ]:
print(oncounts,offcounts)

### excess

In [ ]:
excess = oncounts - offcounts

In [ ]:
excess

### background check

Let's check that there is no excess at larger $\theta^2$ values, where we do not expect any emission.

In [ ]:
x[1:] > 0.05

In [ ]:
check_on = n[x[1:]>0.05].sum()

In [ ]:
check_off = n_off[x[1:]>0.05].sum()

In [ ]:
print('{} - {} = {}'.format(check_on, check_off, check_on - check_off))

### Significance

In [ ]:
total = oncounts + offcounts

In [ ]:
total

In [ ]:
from numpy import sqrt

In [ ]:
sigma = sqrt(total)

In [ ]:
sigma

In [ ]:
significance_1 = excess/sigma

In [ ]:
significance_1

In [ ]:
from gammapy.stats import WStatCountsStatistic

In [ ]:
stat = WStatCountsStatistic(n_on=oncounts, n_off=offcounts, alpha=1.)

print('excess: {} \nsignificance: {}'.format(stat.n_sig,stat.sqrt_ts))

Let's check the background:

In [ ]:
stat_check = WStatCountsStatistic(n_on=check_on, n_off=check_off, alpha=1.)

print('excess: {} \nsignificance: {}'.format(stat_check.n_sig,stat_check.sqrt_ts))

### Your playground

If you want to do it yourself, try the following.

Make a theta^2 plot for signal and background for the second observation run. Calculate the number of on and off events for a theta^2 cut of 0.02. Calculate the significance.

In [ ]:
## your code here

### Combine all runs
We will no use some gammapy code to prepare the $\theta^2$ plot for all runs combined. First we will define the x-axis:

In [ ]:
from gammapy.maps import MapAxis

In [ ]:
theta2_axis = MapAxis.from_bounds(0, 0.1, 
                                  nbin=40, 
                                  unit='deg2'
                                 )

In [ ]:
theta2_axis.edges

In [ ]:
from gammapy.makers.utils import make_theta_squared_table

In [ ]:
theta2_table = make_theta_squared_table(observations=observations,
                                        position=source_pos,
                                        theta_squared_axis=theta2_axis
                                       )

In [ ]:
theta2_table

In [ ]:
all_on = theta2_table['counts'][theta2_table['theta2_max'] < 0.01].sum()

In [ ]:
all_on

In [ ]:
all_off = theta2_table['counts_off'][theta2_table['theta2_max'] < 0.01].sum()

In [ ]:
all_off

In [ ]:
all_excess = all_on - all_off

In [ ]:
all_excess

In [ ]:
stat_all = WStatCountsStatistic(n_on=all_on, n_off=all_off, alpha=1.)

print('excess: {} \nsignificance: {}'.format(stat_all.n_sig,stat_all.sqrt_ts))

In [ ]:
theta2_centre = theta2_axis.center.value

In [ ]:
plt.errorbar(theta2_centre,
             theta2_table['counts'],
             sqrt(theta2_table['counts']),
             label = 'on',
             ls = 'none',
             marker = 'o'
            )

plt.errorbar(theta2_centre,
             theta2_table['counts_off'],
             sqrt(theta2_table['counts_off']),
             label = 'off',
             ls = 'none',
             marker = 'o'
            )

plt.legend()

plt.xlabel('$\\theta^2$ [deg$^2$]')

#plt.savefig('Theta2_all_onoff.svg')

In [ ]:
plt.errorbar(theta2_centre,
             theta2_table['excess'],
             [theta2_table['excess_errn'].data, theta2_table['excess_errp'].data],
             ls = 'none',
             marker = 'o'
            )

plt.xlabel('$\\theta^2$ [deg$^2$]')

#plt.savefig('Theta2_all_excess.svg')

In [ ]:
from gammapy.visualization import plot_theta_squared_table

In [ ]:
plot_theta_squared_table(theta2_table)

## Simple Counts Map

In [ ]:
from gammapy.maps import Map

In [ ]:
map_crab = Map.create(binsz=0.01*u.deg, 
                      width=(5*u.deg, 5*u.deg), 
                      skydir=source_pos, 
                      frame='icrs')

In [ ]:
map_crab.plot(add_cbar = True)

In [ ]:
events.radec

In [ ]:
map_crab.fill_by_coord(events.radec)

In [ ]:
map_crab.plot(
    add_cbar = True,
#    cmap = 'plasma'
)

Find your favourite colour map here : [https://matplotlib.org/stable/users/explain/colors/colormaps.html](https://matplotlib.org/stable/users/explain/colors/colormaps.html)

In [ ]:
smoothed = map_crab.smooth(width=0.05 * u.deg, kernel="gauss")

In [ ]:
smoothed.plot(
    add_cbar=True,
#    cmap = 'plasma'
#    stretch="log", 
)

In [ ]:
smoothed_1 = map_crab.smooth(width=0.2 * u.deg, kernel="gauss")

In [ ]:
smoothed_1.plot(
    add_cbar=True,
#    cmap = 'plasma'
#    stretch="log", 
)

#### Your playground
Make a smoothed sky map. Make the map 3degx3deg large, and use a binning of 0.005deg. Smooth with a disk kernel with 0.03deg. Add a colour bar and stretch such that both the source and the background is clearly seen. You can also try different colour schemes, for example cmap='ocean_r'.

In [ ]:
## your code here

## Summary

In this notebook we have seen how to download and select the data. We had a first look at the information found in the events list and how to make simple plots.